**Importing packages**

In [17]:
import pandas as pd
import sqlite3
db_file = "sqlite_db_pythonsqlite.db"
con = sqlite3.connect(db_file)

# SQL Project Tier 1 Part 1
## Questions

**Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.**

In [19]:
query1 = """ 
SELECT name FROM Facilities
WHERE membercost > 0.0;"""
pd.read_sql_query(query1,con)

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


**Q2: How many facilities do not charge a fee to members?**

In [20]:
query2 = """
SELECT COUNT(name) as No_charge
FROM Facilities
WHERE membercost =0.0;
"""
pd.read_sql_query(query2,con)

,No_charge
0,4


**Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.**

In [21]:
query3 = """
SELECT facid, name,membercost, guestcost, monthlymaintenance
FROM Facilities
WHERE membercost > 0.0
AND membercost < 0.2*monthlymaintenance;
"""
pd.read_sql_query(query3,con)

,facid,name,membercost,guestcost,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,200
1,1,Tennis Court 2,5.0,25.0,200
2,4,Massage Room 1,9.9,80.0,3000
3,5,Massage Room 2,9.9,80.0,3000
4,6,Squash Court,3.5,17.5,80


**Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.**

In [22]:
query4 = """
SELECT * FROM Facilities
WHERE facid in (1,5);
"""
pd.read_sql_query(query4,con)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


**Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $ $$100. Return the name and monthly maintenance of the facilities
in question.**

In [23]:
query5 = """
SELECT `name`, `monthlymaintenance`, 
CASE WHEN `monthlymaintenance`> 100 THEN 'expensive'
ELSE 'cheap' END AS price_range
FROM Facilities;
"""
pd.read_sql_query(query5,con)

,name,monthlymaintenance,price_range
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


**Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.**

In [24]:
query6 = """
SELECT firstname, surname FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members);
"""
pd.read_sql_query(query6,con)

,firstname,surname
0,Darren,Smith


**Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.**

In [25]:
query7 = """
SELECT 
sub.court, 
(sub.firstname || sub.surname) AS name
FROM (SELECT f.name as court, m.firstname AS firstname,m.surname AS surname
FROM Bookings AS b
INNER JOIN Facilities AS f
ON f.facid = b.facid
AND f.name LIKE "Tennis Court%"
INNER JOIN Members AS m
ON b.memid = m.memid) AS sub
GROUP BY sub.firstname,sub.surname
ORDER BY name
"""
pd.read_sql_query(query7,con)

,court,name
0,Tennis Court 1,AnneBaker
1,Tennis Court 2,BurtonTracy
2,Tennis Court 1,CharlesOwen
3,Tennis Court 2,DarrenSmith
4,Tennis Court 1,DavidFarrell
5,Tennis Court 2,DavidJones
6,Tennis Court 1,DavidPinker
7,Tennis Court 1,DouglasJones
8,Tennis Court 1,EricaCrumpet
9,Tennis Court 2,FlorenceBader


**Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $ $$30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.**

In [29]:
query8 = """
SELECT Facilities.name AS facility, (Members.firstname || ' ' || Members.surname) AS name, 
CASE WHEN Members.memid = 0 THEN Facilities.guestcost * Bookings.slots
ELSE Facilities.membercost * Bookings.slots 
END AS cost

FROM Bookings

Inner JOIN Members ON Bookings.memid=Members.memid

Inner JOIN Facilities ON Bookings.facid=Facilities.facid
AND Bookings.starttime LIKE '2012-09-14%'
AND (((Bookings.memid =0) AND (Facilities.guestcost * Bookings.slots >30))
OR ((Bookings.memid !=0) AND (Facilities.membercost * Bookings.slots >30)))

ORDER BY cost DESC;
"""
pd.read_sql_query(query8,con)

,facility,name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


**Q9: This time, produce the same result as in Q8, but using a subquery.**

In [30]:
query9 = """
SELECT * FROM 
(SELECT Facilities.name AS facility, (Members.firstname || ' ' || Members.surname) AS name, 

CASE WHEN Members.memid = 0 THEN Facilities.guestcost * Bookings.slots
ELSE Facilities.membercost * Bookings.slots 
END AS cost

FROM Bookings

Inner JOIN Members ON Bookings.memid=Members.memid

Inner JOIN Facilities ON Bookings.facid=Facilities.facid
AND Bookings.starttime LIKE '2012-09-14%') sub

WHERE sub.cost >30

ORDER BY sub.cost DESC;
"""
pd.read_sql_query(query9,con)


,facility,name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


# SQL Project Tier 1 Part 2

## Local SQL Connection.py import and questions 10,11 and 12


In [32]:

import sqlite3
from sqlite3 import Error
import pandas as pd

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
# /* Q10:Produce a list of facilities with a total revenue less than 1000.
    query10 = """
        SELECT * 
        FROM (
            SELECT sub.facility, SUM( sub.cost ) AS total_revenue
            FROM (
                SELECT Facilities.name AS facility, 
                    CASE WHEN Bookings.memid =0
                    THEN Facilities.guestcost * Bookings.slots
                    ELSE Facilities.membercost * Bookings.slots
                    END AS cost
                FROM Bookings
                
                INNER JOIN Facilities ON Bookings.facid = Facilities.facid
                INNER JOIN Members ON Bookings.memid = Members.memid
                )sub
            GROUP BY sub.facility
        )sub2
        WHERE sub2.total_revenue <1000;
        """
    
# /* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */
    query11 = """
        SELECT a.firstname, a.surname, b.surname,b.firstname
        FROM Members a
        INNER JOIN Members b
        ON a.memid = b.recommendedby
        ORDER BY b.surname,b.firstname;
        """
    
#/* Q12: Find the facilities with their usage by member, but not guests */
    query12 = """
        SELECT Facilities.name, Count(memid) AS usage_by_member
        FROM Facilities
        INNER JOIN Bookings
        ON Facilities.facid = Bookings.facid
        WHERE memid > 0
        GROUP BY Facilities.name; 
        """
# /* Q13: Find the facilities usage by month, but not guests */
    query13 = """
        SELECT b.month, COUNT( b.memid ) AS mem_usage
        FROM (
            SELECT EXTRACT(MONTH FROM starttime) AS month, memid
            FROM Bookings
            WHERE memid !=0
        ) AS b
        GROUP BY b.month; 
        """

    cur.execute(query12)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()


2.6.0
2. Query all tasks
('Badminton Court', 344)
('Massage Room 1', 421)
('Massage Room 2', 27)
('Pool Table', 783)
('Snooker Table', 421)
('Squash Court', 195)
('Table Tennis', 385)
('Tennis Court 1', 308)
('Tennis Court 2', 276)
